In [1]:
#importing libraries for data analysis and data visulisation
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

#importing libraries for calling files 
import os

# Importing Deep Learning Libraries

from keras.preprocessing.image import load_img, img_to_array #model will not take the image it will take array form 
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Input,Dropout,GlobalAveragePooling2D,Flatten,Conv2D,BatchNormalization,Activation,MaxPooling2D # important layers which are required to made the CNN model 
from keras.models import Model,Sequential
from tensorflow.keras.optimizers import Adam, SGD, RMSprop


# Displaying Images

In [15]:
picture_size = 48 # standarised the size of each picture
folder_path = "../input/face-expression-recognition-dataset/images/"

# Ploting the images of disgusting faces

In [17]:
expression = 'disgust'
plt.style.use('dark_background')
plt.figure(figsize= (12,12))
for i in range(1, 10, 1):
    plt.subplot(3,3,i)
    img = load_img(folder_path+"train/"+expression+"/"+
                  os.listdir(folder_path + "train/" + expression)[i], target_size=(picture_size, picture_size))
    plt.imshow(img)   
plt.show()

# Making Training and Validating the data

In [18]:
batch_size  = 128 #signifies that in 1 iteration it should take 128 running examples

datagen_train  = ImageDataGenerator()
datagen_val = ImageDataGenerator()

train_set = datagen_train.flow_from_directory(folder_path+"train",
                                              target_size = (picture_size,picture_size),
                                              color_mode = "grayscale",
                                              batch_size=batch_size,
                                              class_mode='categorical',#for 7 different categories of output
                                              shuffle=True)

#validation set
test_set = datagen_val.flow_from_directory(folder_path+"validation",
                                              target_size = (picture_size,picture_size),
                                              color_mode = "grayscale",
                                              batch_size=batch_size,
                                              class_mode='categorical',
                                              shuffle=False)

# Model Building

In [19]:
no_of_classes = 7

model = Sequential()

#1st CNN layer
model.add(Conv2D(64,(3,3),padding = 'same',input_shape = (48,48,1)))#64 filters and 3x3 kernal size
model.add(BatchNormalization())#Batch normalization applies a transformation that maintains the mean output close to 0
model.add(Activation('relu'))# activation function to increase the non-linearity in the output
model.add(MaxPooling2D(pool_size = (2,2)))#extract the particular imporatant information from that area where this pool is set i.e 2x2
model.add(Dropout(0.25))#to prevent over model from overfitting

#2nd CNN layer
model.add(Conv2D(128,(5,5),padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout (0.25))

#3rd CNN layer
model.add(Conv2D(512,(3,3),padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout (0.25))

#4th CNN layer
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())#for converting the data into a 1-dimensional array for inputting it to the next layer

#Fully connected 1st layer
model.add(Dense(256))#receives input from all the neurons of previous layer
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))


# Fully connected layer 2nd layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(no_of_classes, activation='softmax'))



opt = Adam(lr = 0.0001)#adam optimizer with learning rate 0.0001
model.compile(optimizer=opt,loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()#printing the summary of my model

# Fitting the Model with Training and Validation Data 

In [21]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint("./face_emotion_model.h5", monitor='val_acc', verbose=1, save_best_only=True, mode='max')

early_stopping = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True
                          )

#Reduce the learning rate which was initally set by us when a metric has stopped improving
reduce_learningrate = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

#list containing all three parameters
callbacks_list = [early_stopping,checkpoint,reduce_learningrate]

#epochs defines the number times that the learning algorithm will work through the entire training dataset.
epochs = 48

model.compile(loss='categorical_crossentropy',
              optimizer = Adam(lr=0.001),
              metrics=['accuracy'])

In [22]:
history = model.fit_generator(generator=train_set,
                                steps_per_epoch=train_set.n//train_set.batch_size,
                                epochs=epochs,
                                validation_data = test_set,
                                validation_steps = test_set.n//test_set.batch_size,
                                callbacks=callbacks_list
                                )

# Plotting Accuracy & Loss

In [23]:
plt.style.use('dark_background')

plt.figure(figsize=(20,10))
plt.subplot(1, 2, 1)
plt.suptitle('Optimizer : Adam', fontsize=10)
plt.ylabel('Loss', fontsize=16)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend(loc='upper right')

plt.subplot(1, 2, 2)
plt.ylabel('Accuracy', fontsize=16)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend(loc='lower right')
plt.show()